In [10]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
# Import libraries
import cv2
import tensorflow as tf
import numpy as np

In [12]:
# Load the model
detect_fn = tf.saved_model.load('/content/drive/MyDrive/ssd_mobilenet_v2/saved_model')

# Load category index to map IDs to labels

category_index = {
  16: {'id': 16, 'name': 'bird'}
}  # Fill in based on model's label map

In [ ]:
# Path to your video

video_path = '/content/drive/MyDrive/penguins_in_zoo.mp4'
cap = cv2.VideoCapture(video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Output Video writer
out = cv2.VideoWriter('output_ssd.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

#Process the video frame-by-frame
success, frame = cap.read()
while success:
    # Convert frame to a tensor
    input_tensor = tf.convert_to_tensor([frame], dtype=tf.uint8)
    detections = detect_fn(input_tensor)

    # Extract detection fields
    bboxes = detections['detection_boxes'][0].numpy()      # [N, 4]
    classes = detections['detection_classes'][0].numpy().astype(int)  # [N]
    scores = detections['detection_scores'][0].numpy()     # [N]

    # Draw detections on the frame
    for i, score in enumerate(scores):
        if score > 0.4:  # threshold to filter out low-confidence detections
            (ymin, xmin, ymax, xmax) = bboxes[i]
            (left, right, top, bottom) = (int(xmin * width), int(xmax * width),
                                          int(ymin * height), int(ymax * height))

            class_id = classes[i]
            class_name = category_index[class_id]['name'] if class_id in category_index else 'N/A'

            # Draw bounding box
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.putText(frame, f'{class_name}: {score:.2f}', (left, top - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Write the processed frame to the output video
    out.write(frame)
    success, frame = cap.read()

# Release resources
cap.release()
out.release()


In [ ]:
# Once done, you can display the processed video in Colab
from IPython.display import Video
Video('output_ssd.mp4', width=600, height=400, embed=True)